In [ ]:
# ====================================================================
# 🌌 QEPC UNIVERSAL LOADER
# ====================================================================
import sys, os
# Add the project root (one level up) to the path so we can import the context
sys.path.append(os.path.abspath('..')) 

# Load everything from the central context file
from notebook_context import * # Trigger the success message
print_ready_message()
print("="*80)

-----------------------------------------
---------------------------------------

In [ ]:
from qepc.utils.diagnostics import run_system_check

report = run_system_check()


In [ ]:
import qepc_autoload as qa

# Load the full NBA schedule from data/Games.csv
schedule = qa.load_nba_schedule()  # <= no arguments

print("Number of games in schedule:", len(schedule))
schedule.head()


In [ ]:
import qepc_autoload as qa

today_games = qa.get_today_games()

print("Number of games today:", len(today_games))
today_games


In [ ]:
from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

team_strengths = calculate_advanced_strengths()

print("Number of teams in strength table:", len(team_strengths))
team_strengths.head()
from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

team_strengths = calculate_advanced_strengths()

print("Number of teams in strength table:", len(team_strengths))
team_strengths.head()


In [ ]:
games_to_model = today_games.copy()
print("Using games today:", len(games_to_model))
games_to_model


In [ ]:
from qepc.core.lambda_engine import compute_lambda

lambda_df = compute_lambda(games_to_model, team_strengths)

# Show the key columns
lambda_df[
    ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
].head()


In [ ]:
from qepc.core.simulator import run_qepc_simulation

sim_results = run_qepc_simulation(lambda_df, num_trials=20000)

print("Columns in results:")
print(sim_results.columns.tolist())

sim_results.head()


In [ ]:
# Use the first 4 games in the schedule for testing
games_to_model = schedule.head(4).copy()
print("Using these games:")
games_to_model


In [ ]:
from qepc.core.lambda_engine import compute_lambda

lambda_df = compute_lambda(games_to_model, team_strengths)

lambda_df[
    ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
]


In [ ]:
from qepc.core.simulator import run_qepc_simulation

sim_results = run_qepc_simulation(lambda_df, num_trials=20000)

print("Columns in results:")
print(sim_results.columns.tolist())

sim_results.head()


In [ ]:
from qepc.sports.nba.data_source import load_team_stats

# If you're on local JupyterLab and want to TRY the NBA API first:
team_stats = load_team_stats(prefer_api=True, season="2024-25")

print("Rows in team_stats:", len(team_stats))
team_stats.head()


In [ ]:
import os
os.environ["QEPC_OFFLINE"] = "1"

from qepc.sports.nba.data_source import load_team_stats

team_stats = load_team_stats(prefer_api=True, season="2024-25")
print("Rows in team_stats:", len(team_stats))
team_stats.head()


In [ ]:
pip install nba_api

In [19]:
from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

advanced_strengths = calculate_advanced_strengths()
print("Advanced strengths rows:", len(advanced_strengths))
advanced_strengths.head()


[QEPC Strength V2] Starting Advanced Calculation (Cutoff: Now)...
[QEPC PlayerData] Successfully loaded 1635462 rows from PlayerStatistics.csv.
[QEPC Opponent Processor] Loading raw team data for Weighted DRtg...
[QEPC Opponent Processor] Calculated Weighted DRtg for 30 teams.
[QEPC Strength V2] Calculated Time-Travel Strengths for 30 teams.
Advanced strengths rows: 30


,Team,ORtg,DRtg,Pace,Volatility
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725
1,Boston Celtics,122.0,107.711196,68.080000,12.410859
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424


In [20]:
from qepc.sports.nba.data_source import load_team_stats

team_stats_simple = load_team_stats(prefer_api=True, season="2024-25")
print("Simple team stats rows:", len(team_stats_simple))
team_stats_simple.head()


Simple team stats rows: 6


,Team,GP,ORtg,DRtg
0,Los Angeles Lakers,82,114.5,112.0
1,Golden State Warriors,82,116.0,115.5
2,Boston Celtics,82,118.0,111.0
3,New York Knicks,82,110.0,109.5
4,Chicago Bulls,82,108.0,114.0


In [21]:
# Make sure there's a 'Team' column
if "Team" not in advanced_strengths.columns:
    raise ValueError("Expected a 'Team' column in advanced_strengths")

# If there are multiple rows per team, average them; if it's already unique, this is basically a no-op
advanced_team_strengths = (
    advanced_strengths
    .groupby("Team", as_index=False)
    .mean(numeric_only=True)
)

print("Unique teams in advanced strengths:", len(advanced_team_strengths))
advanced_team_strengths.head()


Unique teams in advanced strengths: 30


,Team,ORtg,DRtg,Pace,Volatility
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725
1,Boston Celtics,122.0,107.711196,68.080000,12.410859
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424


In [22]:
from qepc.core.lambda_engine import compute_lambda
from qepc.core.simulator import run_qepc_simulation

# 1) Pick some games to model — you already had `schedule`
games_to_model = schedule.head(4).copy()   # or today_games if that worked
print("Using these games:")
display(games_to_model)

# 2) Use your advanced strengths (the 30-team table you just checked)
lambda_df = compute_lambda(games_to_model, advanced_team_strengths)

display(
    lambda_df[
        ["Away Team", "Home Team", "lambda_away", "lambda_home", "Volatility_away", "Volatility_home"]
    ]
)

# 3) Run Poisson sims
sim_results = run_qepc_simulation(lambda_df, num_trials=20000)

print("Result columns:")
print(sim_results.columns.tolist())
sim_results.head()


Using these games:


,Date,Time,Away Team,Home Team,Venue,Notes,gameDate
0,10/21/2025,7:30 PM,Houston Rockets,Oklahoma City Thunder,Paycom Center,Regular Season,2025-10-21 19:30:00
1,10/21/2025,10:00 PM,Golden State Warriors,Los Angeles Lakers,Crypto.com Arena,Regular Season,2025-10-21 22:00:00
2,10/22/2025,7:00 PM,Brooklyn Nets,Charlotte Hornets,Spectrum Center,Regular Season,2025-10-22 19:00:00
3,10/22/2025,7:00 PM,Cleveland Cavaliers,New York Knicks,Madison Square Garden,Regular Season,2025-10-22 19:00:00


[QEPC Lambda] Computed lambda & volatility for 4 games.


KeyError: "['Volatility_away', 'Volatility_home'] not in index"

In [23]:
lambda_df.columns.tolist()


['Date',
 'Time',
 'Away Team',
 'Home Team',
 'Venue',
 'Notes',
 'gameDate',
 'lambda_home',
 'lambda_away',
 'vol_home',
 'vol_away']

In [24]:
display(
    lambda_df[
        ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
    ]
)


,Away Team,Home Team,lambda_away,lambda_home,vol_away,vol_home
0,Houston Rockets,Oklahoma City Thunder,86.559122,95.912168,32.325436,14.033294
1,Golden State Warriors,Los Angeles Lakers,99.674868,100.121584,10.256705,13.236314
2,Brooklyn Nets,Charlotte Hornets,106.314731,111.540341,9.097107,13.022697
3,Cleveland Cavaliers,New York Knicks,102.894360,106.596302,12.175616,13.069948


In [25]:
display(
    lambda_df[
        ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
    ]
)


,Away Team,Home Team,lambda_away,lambda_home,vol_away,vol_home
0,Houston Rockets,Oklahoma City Thunder,86.559122,95.912168,32.325436,14.033294
1,Golden State Warriors,Los Angeles Lakers,99.674868,100.121584,10.256705,13.236314
2,Brooklyn Nets,Charlotte Hornets,106.314731,111.540341,9.097107,13.022697
3,Cleveland Cavaliers,New York Knicks,102.894360,106.596302,12.175616,13.069948


In [26]:
from qepc.core.simulator import run_qepc_simulation

sim_results = run_qepc_simulation(lambda_df, num_trials=20000)

sim_results.head()


[QEPC Simulator] Running 20000 trials for 4 games (Chaos Engine Active)...
[QEPC Simulator] Simulation complete.


,Date,Time,Away Team,Home Team,Venue,Notes,gameDate,lambda_home,lambda_away,vol_home,vol_away,Home_Win_Prob,Away_Win_Prob,Tie_Prob,Expected_Score_Total,Expected_Spread,Sim_Home_Score,Sim_Away_Score
0,10/21/2025,7:30 PM,Houston Rockets,Oklahoma City Thunder,Paycom Center,Regular Season,2025-10-21 19:30:00,95.912168,86.559122,14.033294,32.325436,0.65740,0.32680,0.01580,182.74965,9.31305,96.03135,86.71830
1,10/21/2025,10:00 PM,Golden State Warriors,Los Angeles Lakers,Crypto.com Arena,Regular Season,2025-10-21 22:00:00,100.121584,99.674868,13.236314,10.256705,0.50075,0.47600,0.02325,199.83760,0.46030,100.14895,99.68865
2,10/22/2025,7:00 PM,Brooklyn Nets,Charlotte Hornets,Spectrum Center,Regular Season,2025-10-22 19:00:00,111.540341,106.314731,13.022697,9.097107,0.60890,0.36900,0.02210,217.76320,5.16140,111.46230,106.30090
3,10/22/2025,7:00 PM,Cleveland Cavaliers,New York Knicks,Madison Square Garden,Regular Season,2025-10-22 19:00:00,106.596302,102.894360,13.069948,12.175616,0.57650,0.39975,0.02375,209.39395,3.73175,106.56285,102.83110


In [27]:
sim_results[
    ["Away Team", "Home Team", "Home_Win_Prob", "Away_Win_Prob"]
].head()


,Away Team,Home Team,Home_Win_Prob,Away_Win_Prob
0,Houston Rockets,Oklahoma City Thunder,0.65740,0.32680
1,Golden State Warriors,Los Angeles Lakers,0.50075,0.47600
2,Brooklyn Nets,Charlotte Hornets,0.60890,0.36900
3,Cleveland Cavaliers,New York Knicks,0.57650,0.39975


In [28]:
import pandas as pd
from pathlib import Path
from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

# 1) Base strengths
advanced_strengths = calculate_advanced_strengths()
advanced_team_strengths = (
    advanced_strengths
    .groupby("Team", as_index=False)
    .mean(numeric_only=True)
)

# 2) Load your manual injury overrides
project_root = Path(r"C:\Users\wdors\qepc_project")  # adjust if needed
inj_path = project_root / "data" / "Injury_Overrides.csv"
injuries = pd.read_csv(inj_path)

injuries


[QEPC Strength V2] Starting Advanced Calculation (Cutoff: Now)...
[QEPC PlayerData] Successfully loaded 1635462 rows from PlayerStatistics.csv.
[QEPC Opponent Processor] Loading raw team data for Weighted DRtg...
[QEPC Opponent Processor] Calculated Weighted DRtg for 30 teams.
[QEPC Strength V2] Calculated Time-Travel Strengths for 30 teams.


,Team,PlayerName,Status,Impact,Note
0,Indiana Pacers,Tyrese Haliburton,out,0.85,Out for season


In [29]:
team_injury_impact = (
    injuries
    .groupby("Team", as_index=False)["Impact"]
    .prod()
    .rename(columns={"Impact": "InjuryImpact"})
)

team_injury_impact


,Team,InjuryImpact
0,Indiana Pacers,0.85


In [30]:
inj_adjusted = advanced_team_strengths.merge(
    team_injury_impact,
    on="Team",
    how="left"
)

inj_adjusted["InjuryImpact"] = inj_adjusted["InjuryImpact"].fillna(1.0)

inj_adjusted["ORtg_inj"] = inj_adjusted["ORtg"] * inj_adjusted["InjuryImpact"]

inj_adjusted.head()


,Team,ORtg,DRtg,Pace,Volatility,InjuryImpact,ORtg_inj
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725,1.0,122.0
1,Boston Celtics,122.0,107.711196,68.080000,12.410859,1.0,122.0
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107,1.0,122.0
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697,1.0,122.0
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424,1.0,122.0


In [31]:
team_strengths_for_lambda = inj_adjusted.copy()
team_strengths_for_lambda["ORtg"] = team_strengths_for_lambda["ORtg_inj"]

for col in ["ORtg_inj", "InjuryImpact"]:
    if col in team_strengths_for_lambda.columns:
        team_strengths_for_lambda.drop(columns=[col], inplace=True)

team_strengths_for_lambda.head()


,Team,ORtg,DRtg,Pace,Volatility
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725
1,Boston Celtics,122.0,107.711196,68.080000,12.410859
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424


In [32]:
team_strengths_for_lambda = inj_adjusted.copy()
team_strengths_for_lambda["ORtg"] = team_strengths_for_lambda["ORtg_inj"]

for col in ["ORtg_inj", "InjuryImpact"]:
    if col in team_strengths_for_lambda.columns:
        team_strengths_for_lambda.drop(columns=[col], inplace=True)

team_strengths_for_lambda.head()


,Team,ORtg,DRtg,Pace,Volatility
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725
1,Boston Celtics,122.0,107.711196,68.080000,12.410859
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424
